In [1]:
import os
os.environ["LVMCORE_DIR"] = "/Users/droryn/prog/lvm/lvmcore"
os.environ["LVM_MASTER_DIR"] = "/Users/droryn/work/LVM/data/sas/sdsswork/lvm/sandbox/calib"
os.environ["SAS_BASE_DIR"] = "/Users/droryn/work/LVM/data/sas"
#os.environ["LVM_DATA_S"] = "/Users/droryn/work/LVM/data/sas/sdsswork/data/lvm/lco"
os.environ["LVM_SPECTRO_REDUX"] = "/Users/droryn/work/LVM/data/redux"
ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")

In [2]:
import numpy as np
from astropy.table import Table
from astropy.io import fits

from lvmdrp import path
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
from lvmdrp.functions import run_quickdrp as qdrp

from lvmdrp.functions import imageMethod

# define environment before importing any DRP related modules

SLITMAP = Table(drp.fibermap.data)

def _parse_ccd_section(section):
    """Parse a CCD section in the format [1:NCOL, 1:NROW] to python tuples"""
    slice_x, slice_y = section.strip("[]").split(",")
    slice_x = list(map(lambda str: int(str), slice_x.split(":")))
    slice_y = list(map(lambda str: int(str), slice_y.split(":")))
    slice_x[0] -= 1
    slice_y[0] -= 1
    return slice_x, slice_y

def do_for_quadrants(image_path, func, *args, **kwargs):
    with fits.open(image_path) as hdul:
        image = hdul['PRIMARY'].data
        error = hdul['ERROR'].data
        mask = hdul['BADPIX'].data
        mask = ~mask*~np.isfinite(image)*error<=0
        ivar = np.where(mask, 1.0/(error**2), 0.0)
        header = hdul[0].header
        q1x, q1y = _parse_ccd_section(header['HIERARCH AMP1 TRIMSEC'])
        q2x, q2y = _parse_ccd_section(header['HIERARCH AMP2 TRIMSEC'])
        q3x, q3y = _parse_ccd_section(header['HIERARCH AMP3 TRIMSEC'])
        q4x, q4y = _parse_ccd_section(header['HIERARCH AMP4 TRIMSEC'])
        f1 = func(image[q1y[0]:q1y[1],q1x[0]:q1x[1]], ivar[q1y[0]:q1y[1],q1x[0]:q1x[1]], *args, **kwargs)
        f2 = func(image[q2y[0]:q2y[1],q2x[0]:q2x[1]], ivar[q2y[0]:q2y[1],q2x[0]:q2x[1]], *args, **kwargs)
        f3 = func(image[q3y[0]:q3y[1],q3x[0]:q3x[1]], ivar[q3y[0]:q3y[1],q3x[0]:q3x[1]], *args, **kwargs)
        f4 = func(image[q4y[0]:q4y[1],q4x[0]:q4x[1]], ivar[q4y[0]:q4y[1],q4x[0]:q4x[1]], *args, **kwargs)

        filtered = image.copy()*0
        filtered[q1y[0]:q1y[1],q1x[0]:q1x[1]] = f1
        filtered[q2y[0]:q2y[1],q2x[0]:q2x[1]] = f2
        filtered[q3y[0]:q3y[1],q3x[0]:q3x[1]] = f3
        filtered[q4y[0]:q4y[1],q4x[0]:q4x[1]] = f4

        return image, filtered


## Guille's flats with the spectrograph enclosure open

In [3]:
MJD = 60171
print(ORIG_MASTER_DIR)

/Users/droryn/work/LVM/data/sas/sdsswork/lvm/sandbox/calib


In [4]:
drp.get_frames_metadata(mjd=MJD, overwrite=True)

[INFO]: removing metadata store at /Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.2dev/0011XX/11111/60171/raw_metadata.hdf5
[INFO]: Creating new metadata store.
[INFO]: going to extract metadata from 181 frames
extracting metadata:   0%|          | 0/181 [00:00<?, ?frame/s]

extracting metadata: 100%|##########| 181/181 [00:30<00:00,  5.93frame/s]
[INFO]: loading/creating metadata store with parameters tileid = 11111, mjd = 60171 and kind = 'raw'
[INFO]: creating metadata store for tileid = 11111 and mjd = 60171 with 181 new rows
[INFO]: writing raw metadata store to disk


,hemi,tileid,mjd,rmjd,imagetyp,spec,camera,expnum,exptime,neon,...,argon,ldls,quartz,quality,qual,stage,status,drpqual,name,tilegrp
0,s,11111,60171,60171,dark,sp2,r2,3109,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-r2-00003109.fits,0011XX
1,s,11111,60171,60171,dark,sp1,r1,3098,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-r1-00003098.fits,0011XX
2,s,11111,60171,60171,dark,sp2,z2,3098,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z2-00003098.fits,0011XX
3,s,11111,60171,60171,dark,sp1,z1,3109,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z1-00003109.fits,0011XX
4,s,11111,60171,60171,dark,sp3,b3,3099,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b3-00003099.fits,0011XX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,s,11111,60171,60171,dark,sp2,z2,3111,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z2-00003111.fits,0011XX
177,s,11111,60171,60171,dark,sp3,r3,3116,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-r3-00003116.fits,0011XX
178,s,11111,60171,60171,dark,sp3,r3,3106,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-r3-00003106.fits,0011XX
179,s,11111,60171,60171,dark,sp1,r1,3111,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-r1-00003111.fits,0011XX


In [4]:
frames_table = md.get_metadata(tileid="*", mjd=MJD)
frames_table.query("imagetyp == 'dark'", inplace=True)
frames_table.sort_values("camera", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]
frames_table

[INFO]: loading/creating metadata store with parameters tileid = '*', mjd = 60171 and kind = 'raw'
[INFO]: found 181 frames in stores
[INFO]: number of frames after filtering 181


,hemi,tileid,mjd,rmjd,imagetyp,spec,camera,expnum,exptime,neon,...,argon,ldls,quartz,quality,qual,stage,status,drpqual,name,tilegrp
137,s,11111,60171,60171,dark,sp1,b1,3106,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b1-00003106.fits,0011XX
66,s,11111,60171,60171,dark,sp1,b1,3111,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b1-00003111.fits,0011XX
121,s,11111,60171,60171,dark,sp1,b1,3114,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b1-00003114.fits,0011XX
56,s,11111,60171,60171,dark,sp1,b1,3108,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b1-00003108.fits,0011XX
122,s,11111,60171,60171,dark,sp1,b1,3104,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b1-00003104.fits,0011XX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,s,11111,60171,60171,dark,sp3,z3,3113,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z3-00003113.fits,0011XX
132,s,11111,60171,60171,dark,sp3,z3,3101,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z3-00003101.fits,0011XX
111,s,11111,60171,60171,dark,sp3,z3,3109,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z3-00003109.fits,0011XX
98,s,11111,60171,60171,dark,sp3,z3,3110,5.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z3-00003110.fits,0011XX


In [5]:
#os.listdir(ORIG_MASTER_DIR)
masters_mjd = qdrp.get_master_mjd(MJD)
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
print(masters_path)

/Users/droryn/work/LVM/data/sas/sdsswork/lvm/sandbox/calib/60142


In [6]:
# define target folder for detrended pixelflats
target_dir = "/Users/droryn/work/LVM/data/redux/pixflats"
os.makedirs(target_dir, exist_ok=True)

dflat_paths = []
for pixflat in frames_table.to_dict("records"):
    flat_path = path.full("lvm_raw", camspec=pixflat["camera"], **pixflat)
    # print(flat_path)
    
    # output path
    cam = pixflat["camera"]
    exp = pixflat["expnum"]
    dflat_path = os.path.join(target_dir, f"lvm-dpixflat-{cam}-{exp}.fits")
    dflat_paths.append(dflat_path)
    
    # calib paths
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{cam}.fits")
    
    #imageMethod.preproc_raw_frame(in_image=flat_path, out_image=dflat_path, replace_with_nan=False)
    #imageMethod.detrend_frame(in_image=dflat_path, out_image=dflat_path, in_bias=mbias_path, reject_cr=False, replace_with_nan=False)


In [7]:
# Stack frames 
frames_table["dflat_path"] = dflat_paths

cam_groups = frames_table.groupby("camera")
for cam in cam_groups.groups:
    dflat_paths_cam = cam_groups.get_group(cam)["dflat_path"]
    
    # define output combined pixelflat path
    mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
    
    #imageMethod.create_master_frame(in_images=dflat_paths_cam, out_image=mflat_path)

## Iterative smoothing

In [13]:
# Create pixel flats
# p preproc, d detrended, m stacked, c final filtered
#
from scipy import ndimage as ndi
import scipy.interpolate


def median_nan(image, ivar, size=31):
    image_tmp = np.where(ivar>0, image, np.NaN)
    return scipy.ndimage.generic_filter(image_tmp, np.nanmedian, size=size)

def filtering(image, ivar, debug=False):

    minflat = 0.001
    min_flat_for_fit_mask = 0.99
    max_flat_for_fit_mask = 1.02

    # initial model
    smooth = median_nan(image, ivar, size=31)
    if debug:
        fits.writeto('testmodel0.fits', smooth, overwrite=True)

    # initial flat by dividing by smoothed image, masking only where we have no data
    flat  =  (ivar>0)*(smooth>minflat)*image/(smooth*(smooth>minflat)+(smooth<=minflat))
    flat  += (smooth<=minflat)|(ivar<=0)  # set flat to 1 where masked
    if debug:
        fits.writeto('testflat0.fits', flat, overwrite=True)

    # dilate the mask, increasing sigma until not too large
    err = np.sqrt(1./(ivar+(ivar==0)))/(smooth*(image>0)+(image<=0))  # error image
    for nsig in [3.,3.5,4.,5.,10.,20.]:
        mask = (flat<(min_flat_for_fit_mask-nsig*err))|(flat>(max_flat_for_fit_mask+nsig*err))
        mask = ndi.binary_dilation(mask)
        frac = np.sum(mask>0)/float(np.sum(ivar>0))
        if frac<0.05 :
            break
    print("Used nsig = {}, frac = {:4.3f}".format(nsig,frac))

    # https://github.com/desihub/desispec/blob/main/bin/desi_compute_pixel_flatfield#L619

    # now start iterating smoothing and filtering the flat, ignoring newly mased pixels in the smoothing
    mask = mask | (ivar==0)
    smooth = median_nan(image, ~mask, size=31)
    flat  =  (ivar>0)*(smooth>minflat)*image/(smooth*(smooth>minflat)+(smooth<=minflat))   # divide by model
    flat  += (smooth<=minflat)|(ivar<=0)  # set flat to 1 where no data

    if debug:
        fits.writeto('testmask.fits', mask.astype(int), overwrite=True)
        fits.writeto('testivar.fits', ivar, overwrite=True)
        fits.writeto('testmodel.fits', smooth, overwrite=True)
        fits.writeto('testflat.fits', flat, overwrite=True)

    return flat#, model

def filter_image(image_path):
    return do_for_quadrants(image_path, filtering)
    #return do_for_quadrants(image_path, median_nan, size=51)

cam_groups = frames_table.groupby("camera")
for cam in cam_groups.groups:
    if cam!='b1':
        continue
    mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
    pixflat_path = os.path.join(target_dir, f"lvm-cpixflat-{cam}.fits")
    filt_path = os.path.join(target_dir, f"filt-lvm-cpixflat-{cam}.fits")

    with fits.open(mflat_path) as hdul:
        print("Reading :", mflat_path)
        
        image, filtered = filter_image(mflat_path)
        flat = image/filtered
        outf = fits.HDUList(fits.PrimaryHDU(filtered))
        print("Writing :", filt_path)
        outf.writeto(filt_path, overwrite=True)
        outf.close()
        out = fits.HDUList(fits.PrimaryHDU(flat))
        print("Writing :", pixflat_path)
        out.writeto(pixflat_path, overwrite=True)
        out.close()


Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-b1.fits
Used nsig = 3.0, frac = 0.006
Used nsig = 3.0, frac = 0.004
Used nsig = 3.0, frac = 0.003
Used nsig = 3.0, frac = 0.003
Writing : /Users/droryn/work/LVM/data/redux/pixflats/filt-lvm-cpixflat-b1.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-b1.fits


## Trivial Pixelflats

In [12]:
# Create pixel flats
# p preproc, d detrended, m stacked, c final filtered
#
# TODO: deal with NaNs by pre-filtering -- scipy.median_filter spreads them around
# TODO: How to generate bad pixel mask?
#
from scipy import ndimage as ndi
from astropy.io import fits
from multiprocessing.pool import ThreadPool as Pool

def filter_image(image_path, size=30, mode='nearest'):
    with fits.open(mflat_path) as hdul:
        image = hdul[0].data
        header = hdul[0].header
        q1x, q1y = _parse_ccd_section(header['HIERARCH AMP1 TRIMSEC'])
        q2x, q2y = _parse_ccd_section(header['HIERARCH AMP2 TRIMSEC'])
        q3x, q3y = _parse_ccd_section(header['HIERARCH AMP3 TRIMSEC'])
        q4x, q4y = _parse_ccd_section(header['HIERARCH AMP4 TRIMSEC'])
        filtered = np.zeros(image.shape, dtype=np.float32)
        filtered[q1y[0]:q1y[1],q1x[0]:q1x[1]] = ndi.median_filter(image[q1y[0]:q1y[1],q1x[0]:q1x[1]], size=size, mode=mode)
        filtered[q2y[0]:q2y[1],q2x[0]:q2x[1]] = ndi.median_filter(image[q2y[0]:q2y[1],q2x[0]:q2x[1]], size=size, mode=mode) 
        filtered[q3y[0]:q3y[1],q3x[0]:q3x[1]] = ndi.median_filter(image[q3y[0]:q3y[1],q3x[0]:q3x[1]], size=size, mode=mode)
        filtered[q4y[0]:q4y[1],q4x[0]:q4x[1]] = ndi.median_filter(image[q4y[0]:q4y[1],q4x[0]:q4x[1]], size=size, mode=mode)
        #filtered = ndi.median_filter(image, size=30, mode='nearest')
        return image, filtered

def job(mflat_path, pixflat_path):
    print("Reading :", mflat_path)
    image, filtered = filter_image(mflat_path, size=50, mode='nearest')
    flat = image/filtered
    flat = np.where((flat>0.01)*(np.isfinite(flat)), flat, 1.0)
    # outf = fits.HDUList(fits.PrimaryHDU(filtered))
    # print("Writing :", filt_path)
    # outf.writeto(filt_path, overwrite=True)
    # outf.close()
    out = fits.HDUList(fits.PrimaryHDU(flat))
    print("Writing :", pixflat_path)
    out.writeto(pixflat_path, overwrite=True)
    out.close()

cam_groups = frames_table.groupby("camera")
with Pool(4) as p:
    for cam in cam_groups.groups:
        mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
        pixflat_path = os.path.join(target_dir, f"lvm-cpixflat-{cam}.fits")
        filt_path = os.path.join(target_dir, f"filt-lvm-cpixflat-{cam}.fits")

        r = p.apply_async(job, (mflat_path, pixflat_path))
    p.close()
    p.join()

Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-b1.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-b1.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-b2.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-b2.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-b3.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-b3.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-r3.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-r3.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-r1.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-r1.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-z2.fits
Writing : /Users/droryn/work/LVM/data/redux/pixflats/lvm-cpixflat-z2.fits
Reading : /Users/droryn/work/LVM/data/redux/pixflats/lvm-mpixflat-z1.fits
Writing : /Users/droryn/work/LVM/data/

## Check by flatfielding original frames

In [11]:
import scipy.ndimage as ndi
# define target folder for detrended pixelflats
target_dir = "/Users/droryn/work/LVM/data/redux/pixflats"

dflat_paths = []
for pixflat in frames_table.to_dict("records"):
    cam = pixflat["camera"]
    exp = pixflat["expnum"]
    dflat_path = os.path.join(target_dir, f"lvm-dpixflat-{cam}-{exp}.fits")    
    mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
    with fits.open(mflat_path) as hdf:
        f = hdf[0].data

    with fits.open(dflat_path) as hdu:
        d = hdu['PRIMARY'].data
    
    d /= f
    m = ndi.median_filter(d, size=31)
    print(dflat_path, f'test-{cam}.fits')
    fits.writeto(f'test-{cam}.fits', d/m, overwrite=True)


/Users/droryn/work/LVM/data/redux/pixflats/lvm-dpixflat-b1-3106.fits test-b1.fits


[ERROR]: Traceback (most recent call last):
  File "/opt/miniconda/envs/lvmdrp/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/31/fxk1ql6s5bx7q3kh6kwpf8v8c5vp86/T/ipykernel_24089/631794175.py", line 18, in <module>
    m = ndi.median_filter(d, size=31)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda/envs/lvmdrp/lib/python3.12/site-packages/scipy/ndimage/_filters.py", line 1588, in median_filter
    return _rank_filter(input, 0, size, footprint, output, mode, cval,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda/envs/lvmdrp/lib/python3.12/site-packages/scipy/ndimage/_filters.py", line 1489, in _rank_filter
    _nd_image.rank_filter(input, rank, footprint, output, mode, cval,
KeyboardInterrupt



## Nick's pixelflats taken in the lab

In [2]:
drp.get_frames_metadata(mjd=59720, overwrite=True)
drp.get_frames_metadata(mjd=59724, overwrite=True)

[INFO]: removing metadata store at /Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.2dev/0011XX/11111/59720/raw_metadata.hdf5
[INFO]: Creating new metadata store.
[INFO]: going to extract metadata from 192 frames
extracting metadata: 100%|##########| 192/192 [00:25<00:00,  7.46frame/s]
[INFO]: loading/creating metadata store with parameters tileid = 11111, mjd = 59720 and kind = 'raw'
[INFO]: creating metadata store for tileid = 11111 and mjd = 59720 with 192 new rows
[INFO]: writing raw metadata store to disk
[INFO]: removing metadata store at /Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/0.1.2dev/0011XX/11111/59724/raw_metadata.hdf5
[INFO]: Creating new metadata store.
[INFO]: going to extract metadata from 90 frames
extracting metadata: 100%|##########| 90/90 [00:08<00:00, 11.11frame/s]
[INFO]: loading/creating metadata store with parameters tileid = 11111, mjd = 59724 and kind = 'raw'
[INFO]: creating metadata store for tileid = 11111 and mjd = 59724 with

,hemi,tileid,mjd,rmjd,imagetyp,spec,camera,expnum,exptime,neon,...,argon,ldls,quartz,quality,qual,stage,status,drpqual,name,tilegrp
0,s,11111,59724,59724,dark,sp2,z2,3869,60.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z2-00003869.fits,0011XX
1,s,11111,59724,59724,dark,sp2,b2,3872,60.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b2-00003872.fits,0011XX
2,s,11111,59724,59724,dark,sp2,b2,3862,9.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b2-00003862.fits,0011XX
3,s,11111,59724,59724,dark,sp2,z2,3856,9.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z2-00003856.fits,0011XX
4,s,11111,59724,59724,dark,sp2,r2,3874,60.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-r2-00003874.fits,0011XX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,s,11111,59724,59724,dark,sp2,r2,3861,9.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-r2-00003861.fits,0011XX
86,s,11111,59724,59724,dark,sp2,r2,3871,60.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-r2-00003871.fits,0011XX
87,s,11111,59724,59724,dark,sp2,z2,3853,9.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-z2-00003853.fits,0011XX
88,s,11111,59724,59724,dark,sp2,b2,3858,9.0,False,...,False,False,False,excellent,0,1,0,0,sdR-s-b2-00003858.fits,0011XX


In [23]:
# define target folder for preprocessed pixelflats
# leave just preprocessed frames for one low, one high count exposure
# https://docs.google.com/spreadsheets/d/103BNxjlZ59Sob3jDO4EN1z6zp2q5YrYA6nTjGlZM6XY/edit#gid=349553156
target_dir = "/Users/droryn/work/LVM/data/redux/nickflats"
os.makedirs(target_dir, exist_ok=True)
MJD=59720

#masters_mjd = qdrp.get_master_mjd(MJD)
masters_mjd = 60142
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
print(masters_path)

frames_table = md.get_metadata(tileid="*", mjd=MJD)
frames_table.query("expnum>=3814 & expnum<=3843", inplace=True)
frames_table.sort_values("camera", inplace=True)
#frames_table

pflat_paths = []
dflat_paths = []
for pixflat in frames_table.to_dict("records"):
    flat_path = path.full("lvm_raw", camspec=pixflat["camera"], **pixflat)
    # print(flat_path)
    
    # output path
    cam = pixflat["camera"]
    exp = pixflat["expnum"]
    pflat_path = os.path.join(target_dir, f"lvm-ppixflat-{cam}-{exp}.fits")
    pflat_paths.append(pflat_path)
    dflat_path = os.path.join(target_dir, f"lvm-dpixflat-{cam}-{exp}.fits")
    dflat_paths.append(dflat_path)
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{cam}.fits")

    imageMethod.preproc_raw_frame(in_image=flat_path, out_image=pflat_path, assume_imagetyp="pixelflat", replace_with_nan=False)
    imageMethod.detrend_frame(in_image=pflat_path, out_image=dflat_path, in_bias=mbias_path, reject_cr=False, replace_with_nan=False)


[INFO]: loading/creating metadata store with parameters tileid = '*', mjd = 59720 and kind = 'raw'
[INFO]: found 192 frames in stores
[INFO]: number of frames after filtering 192
[INFO]: starting preprocessing of raw image 'sdR-s-b2-00003820.fits.gz'


/Users/droryn/work/LVM/data/calib/60142


[WARNING]: assuming IMAGETYP = 'pixelflat'
[INFO]: exposure time 420.0 (s)
[WARNING]: assuming TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[WARNING]: assuming BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.45, 2.46, 2.46, 2.5] (e-/ADU)
[INFO]: masked 4679 (13.49%) pixels in overscan above 3.0 standard deviations
[INFO]: median and standard deviation in OS quadrant 1: 1021.00 +/- 1.09 (ADU)
[INFO]: masked 4519 (13.03%) pixels in overscan above 3.0 standard deviations
[INFO]: median and standard deviation in OS quadrant 2: 1023.00 +/- 1.12 (ADU)
[INFO]: masked 2629 (7.58%) pixels in overscan above 3.0 standard deviations
[INFO]: median and standard deviation in OS quadrant 3: 1021.00 +/- 1.32 (ADU)
[INFO]: masked 2487 (7.17%) pixels in overscan above 3.0 standard deviations
[INFO]: median and standard deviation in OS quadrant 4: 1022.00 +

In [24]:
from astropy.io import fits
with fits.open('/Users/droryn/work/LVM/data/redux/nickflats/lvm-ppixflat-r2-3836.fits') as f1:
    d1 = f1['PRIMARY'].data
with fits.open('/Users/droryn/work/LVM/data/calib/60177/lvm-mpixflat-r2.fits') as f2:
    d2 = f2['PRIMARY'].data

flat = d2

d = d1/flat

fits.writeto('/Users/droryn/work/LVM/data/redux/nickflats/test.fits', d, overwrite=True)


In [2]:
import numpy as np
from scipy import signal
from astropy.io import fits
import matplotlib.pyplot as plt

with fits.open('/Users/droryn/work/LVM/data/sas/sdsswork/data/lvm/lco/59720/sdR-s-r2-00003836.fits.gz') as f:
    d1 = f['PRIMARY'].data
kernel = np.outer(signal.windows.gaussian(300, 50), signal.windows.gaussian(300, 50))
kernel = kernel/np.sum(kernel)
d = signal.fftconvolve(d1, kernel, mode='same')
fits.writeto('/Users/droryn/work/LVM/data/redux/nickflats/test.fits', d, overwrite=True)
